In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model

2024-07-10 13:03:59.472869: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 13:03:59.472974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 13:03:59.617949: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Define paths
trainpath = '/kaggle/input/riceleafdataset/dataset/train'
testpath = '/kaggle/input/riceleafdataset/dataset/test'

# Image Processing - Training Data
new_size = 224
train_images = []
train_labels = []
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

for i in os.listdir(trainpath):
    if i in class_disease:
        print("Entering the folder:", i)
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            train_images.append(list(resize_image))
            train_labels.append(class_disease[i])

# Image Processing - Testing Data
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):
    if i in class_disease:
        print("Entering to the folder name:", i)
        files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(testpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder is", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            test_images.append(list(resize_image))
            test_labels.append(class_disease[i])

def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)


Entering the folder: Tungro
Number of images in the folder: 1108
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BrownSpot
Number of images in the folder: 1400
Entering to the folder name: Tungro
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BrownSpot
Number of images in the folder is 200
(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)


In [3]:
from skimage.feature import hog
from tqdm import tqdm 

def extract_hog_features(images):
    hog_features = []
    for img in tqdm(images, desc="Extracting HOG features"):
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        feature, _ = hog(img, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True)
        hog_features.append(feature)
    return np.array(hog_features)

In [4]:
train_feature_hog = extract_hog_features(X_train)
test_feature_hog = extract_hog_features(X_test)

Extracting HOG features: 100%|██████████| 800/800 [04:29<00:00,  2.96it/s]


# hog + Random Forest Classifier

In [15]:
from sklearn.ensemble import RandomForestClassifier

def train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test):

        rf = RandomForestClassifier()
        rf = rf.fit(train_feature_hog, y_train)
        test_pred = rf.predict(test_feature_hog)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'hog+RF.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.43125000, Recall: 0.43125000, Precision: 0.99764151, F1 Score: 0.58945616, AUC: 0.71541667
Run 2 - Accuracy: 0.42750000, Recall: 0.42750000, Precision: 1.00000000, F1 Score: 0.58800471, AUC: 0.71375000
Run 3 - Accuracy: 0.41750000, Recall: 0.41750000, Precision: 1.00000000, F1 Score: 0.57875971, AUC: 0.70875000
Run 4 - Accuracy: 0.42500000, Recall: 0.42500000, Precision: 1.00000000, F1 Score: 0.58887991, AUC: 0.71250000
Run 5 - Accuracy: 0.43000000, Recall: 0.43000000, Precision: 1.00000000, F1 Score: 0.59240396, AUC: 0.71500000
Run 6 - Accuracy: 0.43500000, Recall: 0.43500000, Precision: 1.00000000, F1 Score: 0.60131883, AUC: 0.71750000
Run 7 - Accuracy: 0.43750000, Recall: 0.43750000, Precision: 1.00000000, F1 Score: 0.60019788, AUC: 0.71875000
Run 8 - Accuracy: 0.42500000, Recall: 0.42500000, Precision: 1.00000000, F1 Score: 0.58802174, AUC: 0.71250000
Run 9 - Accuracy: 0.44375000, Recall: 0.44375000, Precision: 1.00000000, F1 Score: 0.60629683, AUC: 0.72187500
R

# hog + Decision Tree Classifier

In [16]:
from sklearn.tree import DecisionTreeClassifier

def train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test):

        dt = DecisionTreeClassifier()
        dt.fit(train_feature_hog, y_train)
        test_pred = dt.predict(test_feature_hog)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'hog+DTC.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.57750000, Recall: 0.57750000, Precision: 0.57791792, F1 Score: 0.57604971, AUC: 0.71833333
Run 2 - Accuracy: 0.57250000, Recall: 0.57250000, Precision: 0.57273130, F1 Score: 0.57100130, AUC: 0.71500000
Run 3 - Accuracy: 0.59125000, Recall: 0.59125000, Precision: 0.59017494, F1 Score: 0.58844049, AUC: 0.72750000
Run 4 - Accuracy: 0.58875000, Recall: 0.58875000, Precision: 0.59025347, F1 Score: 0.58878016, AUC: 0.72583333
Run 5 - Accuracy: 0.58000000, Recall: 0.58000000, Precision: 0.58085680, F1 Score: 0.57944598, AUC: 0.72000000
Run 6 - Accuracy: 0.59250000, Recall: 0.59250000, Precision: 0.59290847, F1 Score: 0.59199801, AUC: 0.72833333
Run 7 - Accuracy: 0.59875000, Recall: 0.59875000, Precision: 0.59918201, F1 Score: 0.59818726, AUC: 0.73250000
Run 8 - Accuracy: 0.57875000, Recall: 0.57875000, Precision: 0.57767997, F1 Score: 0.57736797, AUC: 0.71916667
Run 9 - Accuracy: 0.60125000, Recall: 0.60125000, Precision: 0.60106435, F1 Score: 0.60096641, AUC: 0.73416667
R

# hog + KNN Classifier

In [17]:
from sklearn.neighbors import KNeighborsClassifier

def train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test):

        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(train_feature_hog, y_train)
        test_pred = knn.predict(test_feature_hog)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'hog+KNN.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.63750000, Recall: 0.63750000, Precision: 0.73384469, F1 Score: 0.63640990, AUC: 0.76541667
Run 2 - Accuracy: 0.63750000, Recall: 0.63750000, Precision: 0.73384469, F1 Score: 0.63640990, AUC: 0.76541667
Run 3 - Accuracy: 0.63750000, Recall: 0.63750000, Precision: 0.73384469, F1 Score: 0.63640990, AUC: 0.76541667
Run 4 - Accuracy: 0.63750000, Recall: 0.63750000, Precision: 0.73384469, F1 Score: 0.63640990, AUC: 0.76541667
Run 5 - Accuracy: 0.63750000, Recall: 0.63750000, Precision: 0.73384469, F1 Score: 0.63640990, AUC: 0.76541667
Run 6 - Accuracy: 0.63750000, Recall: 0.63750000, Precision: 0.73384469, F1 Score: 0.63640990, AUC: 0.76541667
Run 7 - Accuracy: 0.63750000, Recall: 0.63750000, Precision: 0.73384469, F1 Score: 0.63640990, AUC: 0.76541667
Run 8 - Accuracy: 0.63750000, Recall: 0.63750000, Precision: 0.73384469, F1 Score: 0.63640990, AUC: 0.76541667
Run 9 - Accuracy: 0.63750000, Recall: 0.63750000, Precision: 0.73384469, F1 Score: 0.63640990, AUC: 0.76541667
R

# hog + SVM Classifier

In [ ]:
from sklearn.svm import SVC

def train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test):

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        # Define and train SVM Classifier
        svm = SVC(probability=True)
        svm.fit(train_feature_hog, y_train_cat)
        test_pred = svm.predict(test_feature_hog)
        test_pred_proba = svm.predict_proba(test_feature_hog)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'hog+SVM.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.89125000, Recall: 0.89125000, Precision: 0.89220896, F1 Score: 0.88960075, AUC: 0.98907917
Run 2 - Accuracy: 0.89125000, Recall: 0.89125000, Precision: 0.89220896, F1 Score: 0.88960075, AUC: 0.98900417


In [5]:
from xgboost import XGBClassifier

def train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test):

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        xgb.fit(train_feature_hog, y_train_cat)
        test_pred = xgb.predict(test_feature_hog)
        test_pred_proba = xgb.predict_proba(test_feature_hog)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_hog, y_train, test_feature_hog, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'hog+XGB.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.86625000, Recall: 0.86625000, Precision: 0.87183054, F1 Score: 0.86231055, AUC: 0.98116875
Run 2 - Accuracy: 0.86625000, Recall: 0.86625000, Precision: 0.87183054, F1 Score: 0.86231055, AUC: 0.98116875
Run 3 - Accuracy: 0.86625000, Recall: 0.86625000, Precision: 0.87183054, F1 Score: 0.86231055, AUC: 0.98116875
Run 4 - Accuracy: 0.86625000, Recall: 0.86625000, Precision: 0.87183054, F1 Score: 0.86231055, AUC: 0.98116875
Run 5 - Accuracy: 0.86625000, Recall: 0.86625000, Precision: 0.87183054, F1 Score: 0.86231055, AUC: 0.98116875
Run 6 - Accuracy: 0.86625000, Recall: 0.86625000, Precision: 0.87183054, F1 Score: 0.86231055, AUC: 0.98116875
Run 7 - Accuracy: 0.86625000, Recall: 0.86625000, Precision: 0.87183054, F1 Score: 0.86231055, AUC: 0.98116875
Run 8 - Accuracy: 0.86625000, Recall: 0.86625000, Precision: 0.87183054, F1 Score: 0.86231055, AUC: 0.98116875
Run 9 - Accuracy: 0.86625000, Recall: 0.86625000, Precision: 0.87183054, F1 Score: 0.86231055, AUC: 0.98116875
R